# VGG
![](./Picture/vgg.png)

In [7]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def vgg_block(num_convs, in_channels, out_channels):
    block = []
    for i in range(num_convs):
        if i == 0:
            block.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1))
        else:
            block.append(nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1))
        block.append(nn.ReLU())
    block.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*block)


def VGG_11(conv_arch, fc_features, fc_hidden_units):
    net = nn.Sequential()
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        net.add_module(
            "vgg_block_" + str(i + 1), vgg_block(num_convs=num_convs, in_channels=in_channels, out_channels=out_channels)
            )
    net.add_module("fc", 
                   nn.Sequential(d2l.FlattenLayer(),
                                 nn.Linear(fc_features, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, 10)
                                 ))
    return net

In [8]:
conv_arch = ((1, 1, 64//8), (1, 64//8, 128//8), (2, 128//8, 256//8), (2, 256//8, 512//8), (2, 512//8, 512//8))
# 经过5个vgg_block, 宽高会减半5次, 变成 224/32 = 7
fc_features = 512 * 7 * 7 # c * w * h
fc_hidden_units = 4096 # 任意

net = VGG_11(conv_arch=conv_arch, fc_features=fc_features//8, fc_hidden_units=fc_hidden_units//8)
batch_size = 64
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
lr, num_epochs = 0.001, 3
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


training on  cuda
epoch 1, loss 0.5884, train acc 0.779, test acc 0.874, time 88.4 sec
epoch 2, loss 0.3213, train acc 0.884, test acc 0.891, time 87.5 sec
epoch 3, loss 0.2769, train acc 0.901, test acc 0.904, time 87.4 sec
